In [1]:
# libraries
library(Seurat)
library(tidyverse)
library(igraph)
require(circlize)
library(R.utils)
library(data.table) #to read gz file

Attaching SeuratObject

Warning message:
“package ‘tidyverse’ was built under R version 4.1.3”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.4     ✔ purrr   1.0.2
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.3.1     ✔ stringr 1.5.1
✔ readr   2.1.5     ✔ forcats 1.0.0
Warning message:
“package ‘tibble’ was built under R version 4.1.3”
Warning message:
“package ‘forcats’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following o

In [2]:
input_dir <- "../../../../data_preprocessing/Lasry/2.filtering/outs/"

In [3]:
# # load counts
# print("load counts")

counts <- fread(paste0(input_dir,"counts_lognorm.csv.gz"), header = TRUE,check.names=FALSE)
counts <- as.data.frame(counts)
rownames(counts) <- counts$gene_symbol
counts <- counts[,-1]

In [9]:
# load cell annotation
print("load cell annotation")
anno_cells <- read.table(paste0(input_dir,"anno_cells_norm.txt")
                         ,sep = "\t"
                         # ,row.names = 2
                         ,header = TRUE
                         ,check.names=FALSE
                         )
# print(str(anno_cells))

[1] "load cell annotation"


In [10]:
#set rownames of annotation to cell_ids
rownames(anno_cells) <- anno_cells$cell

#set colnames of counts to cell_ids
colnames(counts) <- rownames(anno_cells)

In [11]:
avg_expression <- rowMeans(counts)

In [12]:
gene_names <- rownames(counts)

In [13]:
avg_expr_table <- data.frame(Gene = gene_names, AverageExpression = avg_expression)

In [14]:
case <- anno_cells$health_status == "AML"
case_idx <- rownames(anno_cells)[case]

In [15]:
avg_expression_case <- rowMeans(counts[,case_idx])
# gene_names <- rownames(counts[,case_idx])
# avg_expr_table_case <- data.frame(Gene = gene_names, AverageExpression = avg_expression)

In [16]:
control <- anno_cells$health_status == "healthy"
control_idx <- rownames(anno_cells)[control]

In [17]:
avg_expression_control <- rowMeans(counts[,control_idx])

In [18]:
avg_expr_table$control <- avg_expression_control

In [19]:
avg_expr_table$case <- avg_expression_case

In [20]:
#create a Seurat object
srt=CreateSeuratObject(counts=counts, meta.data=anno_cells)

In [21]:
#peek into the number of cells for case/control
srt@meta.data$health_status %>% table()

.
    AML healthy 
  21311   25391 

In [22]:
#peek into the number of cell types
srt@meta.data$cell_type %>% table()

.
    B    DC   Ery  Gran  HSPC  Mono    NK     T 
 4765  1634  1674  2332  3169 18004  3078 12046 

In [23]:
#set the indent to cell_type
Idents(srt) <- "cell_type"

In [24]:
# initialize empty vector for storing DEGs
DEGs <- c()

# iterate over each unique cell type 
for (cell_type in unique(srt@meta.data$cell_type)) {
  
  # subset Seurat object to only include cells of current cell type
  seurat_obj_receiver <- subset(srt, idents = cell_type)
  
  # set cell identity using the "health_status" feature
  seurat_obj_receiver <- SetIdent(seurat_obj_receiver, value = seurat_obj_receiver[["health_status"]])
  
  # specify the two conditions to compare
  condition_oi <- "AML"
  condition_reference <- "healthy" 
  
#   find differentially expressed genes between the two conditions
  DE_table_receiver <- FindMarkers(object = seurat_obj_receiver, 
                                   slot = "counts",
                                   ident.1 = condition_oi, 
                                   ident.2 = condition_reference, 
                                   min.pct = 0.10) %>%
    # convert row names to a separate "gene" column
    rownames_to_column("gene")
    
#     DE_table_receiver <- FindMarkers(object = seurat_obj_receiver, 
#                                    slot = "data",
#                                    ident.1 = condition_oi, 
#                                    ident.2 = condition_reference, 
#                                    min.pct = -Inf,
#                                    thresh.use = 0,
#                                    logfc.threshold = -Inf,
#                                    min.cells.feature = 1, min.cells.group = 1) %>% rownames_to_column("gene")
    
    
    
  
  # add cell type information to the DEG table
  DE_table_receiver <- data.frame(cluster = cell_type, DE_table_receiver)
  
  # filter DEGs based on statistical significance and fold change threshold
  DE_table_receiver <- DE_table_receiver %>% 
    filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25)
  
  # print cell type and number of DEGs found
  print(cell_type)
  print("number of genes in the cell type")
  print(length(row.names(seurat_obj_receiver@assays$RNA@data)))
  print("number of genes output from DE")
  print(nrow(DE_table_receiver))
  
  # append DEGs to the vector of all DEGs
  DEGs <- rbind(DEGs, DE_table_receiver)
}

[1] "Mono"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 386
[1] "Gran"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 337
[1] "T"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 216
[1] "NK"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 472
[1] "B"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 297
[1] "HSPC"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 452
[1] "Ery"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 1082
[1] "DC"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 239


In [26]:
table(DEGs$cluster)


   B   DC  Ery Gran HSPC Mono   NK    T 
 297  239 1082  337  452  386  472  216 

In [27]:
write.csv(table(DEGs$cluster), "DEGs_numbers.csv")

In [ ]:
write.csv(DEGs, "DEG_significant_table.csv")

In [57]:
write.csv(DEGs, "DEG_table.csv")